In [14]:
# ==============================================================================
# CELL 2A: XỬ LÝ CHUYÊN BIỆT CHO DATASET A (V6 - LẤY CẢ THÔNG TIN CÁ NHÂN)
# ==============================================================================
import pandas as pd
import re

def preprocess_dataset_A_full(file_path: str) -> pd.DataFrame:
    """
    Hàm này xử lý file Excel của Dataset A.
    1. Lấy và đổi tên các cột thông tin cá nhân (c1 -> School, c2 -> Gender, ...).
    2. Tạo tên cột ngữ cảnh cho các câu hỏi tâm lý (ví dụ: 'PERF_LEV_EC1').
    3. Kết hợp cả hai thành một DataFrame hoàn chỉnh.
    """
    try:
        # 1. Đọc cả hai sheet
        data_df = pd.read_excel(file_path, sheet_name='Data', header=0) 
        codes_df = pd.read_excel(file_path, sheet_name='Questionnaires and Codes')
        print(f"Đã đọc thành công file: {file_path}")

        # --- LUỒNG 1: XỬ LÝ THÔNG TIN CÁ NHÂN ---
        # Ánh xạ cứng cho 4 cột đầu tiên
        demographic_rename_map = {
            'c1': 'School',
            'c2': 'Gender',
            'c3': 'YOB', # Year of Birth
            'c4': 'GPA'
        }
        demographic_df = data_df[list(demographic_rename_map.keys())].rename(columns=demographic_rename_map)
        print(f"  -> Đã xử lý {demographic_df.shape[1]} cột thông tin cá nhân.")

        # --- LUỒNG 2: XỬ LÝ THÔNG TIN TÂM LÝ ---
        # 2a. Xây dựng "Từ điển Ánh xạ" thông minh từ cột 'Questions'
        code_mapper = {}
        current_group_prefix = ""
        
        codes_df['question_key'] = codes_df['Questions'].str.extract(r'^(\d+\.\d+)\s')
        
        for index, row in codes_df.iterrows():
            question_key = str(row['question_key'])
            code = str(row['Code'])
            is_group_header = pd.isna(row['question_key']) and pd.notna(code)

            if is_group_header:
                if code not in ['School', 'Gender', 'YOB', 'GPA']:
                    current_group_prefix = code
            elif pd.notna(row['question_key']) and pd.notna(code) and current_group_prefix:
                contextual_code = f"{current_group_prefix}_{code}"
                code_mapper[question_key] = contextual_code

        # 2b. Tạo Từ điển Đổi tên Cột cho DataFrame Dữ liệu
        psychological_rename_mapper = {}
        for col_name in data_df.columns:
            match = re.match(r'c(\d+)x(\d+)', str(col_name))
            if match:
                question_number_str = f"{match.group(1)}.{match.group(2)}"
                if question_number_str in code_mapper:
                    psychological_rename_mapper[col_name] = code_mapper[question_number_str]

        # 2c. Thực hiện đổi tên và chọn lọc cột
        data_df_renamed = data_df.rename(columns=psychological_rename_mapper)
        psychological_columns = list(psychological_rename_mapper.values())
        psychological_df = data_df_renamed[psychological_columns]
        print(f"  -> Đã xử lý {psychological_df.shape[1]} cột dữ liệu tâm lý.")

        # 3. KẾT HỢP HAI LUỒNG
        final_df = pd.concat([demographic_df, psychological_df], axis=1)
        
        print(f"  -> Kết hợp thành công! DataFrame cuối cùng có {final_df.shape[1]} cột.")
        return final_df

    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý file {file_path}: {e}")
        return None

# Sử dụng hàm để xử lý Dataset A
file_a = "data/student_data/Student-Academic-Perfectionism-1.xlsx" 
data_a = preprocess_dataset_A_full(file_a)

if data_a is not None:
    print("\nTiền xử lý Dataset A hoàn tất!")
    print(f"Dataset A có {data_a.shape[0]} dòng và {data_a.shape[1]} cột.")
    print("5 dòng đầu của Dataset A hoàn chỉnh:")
    print(data_a.head())

Đã đọc thành công file: data/student_data/Student-Academic-Perfectionism-1.xlsx
  -> Đã xử lý 4 cột thông tin cá nhân.
  -> Đã xử lý 46 cột dữ liệu tâm lý.
  -> Kết hợp thành công! DataFrame cuối cùng có 50 cột.

Tiền xử lý Dataset A hoàn tất!
Dataset A có 2942 dòng và 50 cột.
5 dòng đầu của Dataset A hoàn chỉnh:
   School  Gender   YOB  GPA  FRE_ACT_Mental_Health  FRE_ACT_Physical_Health  \
0       5       1  2003    3                      1                        5   
1       6       1  2003    3                      1                        3   
2       1       2  2002    4                      1                        3   
3       4       2  2003    3                      1                        4   
4       7       1  2003    3                      1                        5   

   FRE_ACT_Environment  FRE_ACT_Education  FRE_ACT_Gender_Equity  \
0                    4                  1                      3   
1                    4                  2                      2   


In [28]:
# ==============================================================================
# CELL 2B: XỬ LÝ CHUYÊN BIỆT CHO DATASET B (V6 - FINAL)
# ==============================================================================
import pandas as pd

def preprocess_dataset_B_final(file_path: str) -> pd.DataFrame:
    """
    Hàm này được thiết kế đặc biệt để xử lý file Excel của Dataset B.
    Phiên bản này sửa lỗi trùng lặp tên cột bằng cách tách biệt hoàn toàn
    việc xử lý dữ liệu cá nhân và dữ liệu tâm lý.
    """
    try:
        # 1. Đọc cả hai sheet
        data_df = pd.read_excel(file_path, sheet_name='Responses', header=0) 
        codes_df = pd.read_excel(file_path, sheet_name='Questions and Codes')
        print(f"Đã đọc thành công file: {file_path}")

        # --- LUỒNG 1: XỬ LÝ THÔNG TIN CÁ NHÂN (Giữ nguyên) ---
        demographic_data_df = data_df.iloc[:, 0:12]
        # Lấy chính xác 12 mã code từ 12 dòng đầu
        demographic_codes = codes_df.iloc[0:12]['Code'].dropna().tolist()
        demographic_data_df.columns = demographic_codes
        print(f"  -> Đã xử lý {demographic_data_df.shape[1]} cột thông tin cá nhân.")

        # --- LUỒNG 2: XỬ LÝ THÔNG TIN TÂM LÝ (SỬA LỖI LOGIC) ---
        
        # 2a. Cắt DataFrame dữ liệu thô
        start_data_col_index = 12
        psychological_data_df = data_df.iloc[:, start_data_col_index:]
        num_data_columns = psychological_data_df.shape[1]
        
        # 2b. Chuẩn bị "bảng tra cứu" codes_df
        # === SỬA LỖI: Chỉ bắt đầu xử lý từ dòng 14 (index 13) ===
        psychological_codes_df = codes_df.iloc[13:].copy()
        
        # Lọc ra các dòng câu hỏi thực sự
        psychological_codes_df = psychological_codes_df.dropna(subset=['No.'])
        
        # Lấy danh sách mã code gốc
        psychological_codes_list_full = psychological_codes_df['Code'].str.strip().tolist()
        
        # 2c. Cắt danh sách mã code để khớp với số cột dữ liệu
        psychological_codes_list = psychological_codes_list_full[:num_data_columns]
        num_psychological_codes = len(psychological_codes_list)
        
        print(f"  -> Đã tìm thấy {num_data_columns} cột dữ liệu tâm lý.")
        print(f"  -> Đã tạo {num_psychological_codes} mã code tương ứng.")
        
        if num_data_columns != num_psychological_codes:
            # Đoạn này chỉ để phòng hờ, logic mới sẽ không để lỗi này xảy ra
             raise ValueError(f"Số cột vẫn không khớp: {num_data_columns} != {num_psychological_codes}")

        # 2d. Đổi tên cột
        psychological_data_df.columns = psychological_codes_list
        
        # 3. KẾT HỢP HAI LUỒNG
        psychological_data_df.index = demographic_data_df.index
        final_df = pd.concat([demographic_data_df, psychological_data_df], axis=1)
        
        print(f"  -> Kết hợp thành công! DataFrame cuối cùng có {final_df.shape[1]} cột.")
        return final_df

    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý file {file_path}: {e}")
        return None

# Sử dụng hàm để xử lý Dataset B
file_b = "data/student_data/Survey on Vietnamese Students' School Alienation, Academic Contingent Self-worth and Parents' and Teachers' Academic Conditional Regard.xlsx" 
data_b = preprocess_dataset_B_final(file_b)

if data_b is not None:
    print("\nTiền xử lý Dataset B hoàn tất!")
    print(f"Dataset B có {data_b.shape[0]} dòng và {data_b.shape[1]} cột.")
    print("\nCác cột của Dataset B:")
    print(list(data_b.columns))
    print("\n5 dòng đầu của Dataset B hoàn chỉnh:")
    print(data_b.head())

Đã đọc thành công file: data/student_data/Survey on Vietnamese Students' School Alienation, Academic Contingent Self-worth and Parents' and Teachers' Academic Conditional Regard.xlsx
  -> Đã xử lý 12 cột thông tin cá nhân.
  -> Đã tìm thấy 50 cột dữ liệu tâm lý.
  -> Đã tạo 50 mã code tương ứng.
  -> Kết hợp thành công! DataFrame cuối cùng có 62 cột.

Tiền xử lý Dataset B hoàn tất!
Dataset B có 2970 dòng và 62 cột.

Các cột của Dataset B:
['birth', 'grade', 'gender', 'school', 'gpa', 'gpa_des', 'livewith', 'mom_edu', 'mom_occ', 'mom_inc', 'dad_edu', 'dad_occ', 'par_neg_reg', 'par_hard_appr', 'par_exam_acpt', 'par_achv_care', 'par_well_love', 'par_achv_warm', 'par_pos_reg', 'par_lazy_appr_neg', 'par_exam_acpt_neg', 'par_achv_care_neg', 'par_well_love_neg', 'par_achv_warm_neg', 'teach_neg_reg', 'teach_hard_appr', 'teach_exam_acpt', 'teach_achv_care', 'teach_well_like', 'teach_achv_fnd', 'teach_pos_reg', 'teach_hard_appr_neg', 'teach_exam_acpt_neg', 'teach_achv_care_neg', 'teach_well_like

In [32]:
pd.set_option('display.max_columns', None)

In [33]:
data_a

,School,Gender,YOB,GPA,FRE_ACT_Mental_Health,FRE_ACT_Physical_Health,FRE_ACT_Environment,FRE_ACT_Education,FRE_ACT_Gender_Equity,FRE_ACT_Poverty_Eradication,FRE_ACT_Peace_Justice,PERF_LEV_EC1,PERF_LEV_EC1,PERF_LEV_EC2,PERF_LEV_SPP1,PERF_LEV_EC3,PERF_LEV_SPP2,PERF_LEV_EC4,PERF_LEV_PS1,PERF_LEV_SPP3,PERF_LEV_PS2,PERF_LEV_EC1,PERF_LEV_EC1,PERF_LEV_PS3,PERF_LEV_SPP5,PERF_LEV_SPP6,PERF_LEV_PS4,PERF_LEV_SPP7,PERF_LEV_SPP8,PERF_LEV_SPP9,PERF_LEV_SPP10,MERIT_School_Chance,MERIT_Achieve_Factors,MERIT_School_Grade,MERIT_School_Reward,MERIT_School_Deserve,MERIT_School_WorkHard,COMPET_Student_Friend,COMPET_Student_Compete,COMPET_Student_Try,COMPET_Student_Classmate,COMPET_Students_LeftOut,INTRINS_Learn_New,INTRINS_Learn_Selfexp,INTRINS_Learn_Discover,INTRINS_Learn_Accomplish,INTRINS_Learn_Broaden,INTRINS_Learn_Process,INTRINS_Allow_Interest,INTRINS_Allow_Excellent
0,5,1,2003,3,1,5,4,1,3,3,3,4,5,5,3,4,4,4,4,5,3,4,5,5,3,3,5,4,2,3,3,5,2,5,3,4,5,5,5,5,1,4,1,4,3,4,4,4,3,5
1,6,1,2003,3,1,3,4,2,2,3,5,4,3,4,2,3,3,4,2,4,3,4,3,3,4,5,5,5,4,5,4,4,3,4,3,5,4,5,4,4,1,4,3,4,5,4,5,5,3,3
2,1,2,2002,4,1,3,5,1,2,3,4,4,5,3,3,5,4,3,3,4,3,4,5,5,3,5,4,3,2,4,4,5,2,5,4,5,4,5,4,5,1,5,4,5,4,5,5,4,4,4
3,4,2,2003,3,1,4,4,2,3,3,1,4,5,4,2,5,3,4,5,4,5,4,5,4,5,4,5,2,5,4,3,3,3,3,5,4,5,4,5,4,1,4,5,3,5,3,3,5,5,4
4,7,1,2003,3,1,5,3,1,2,4,3,3,4,3,2,3,3,4,4,5,4,3,4,4,3,4,4,3,4,4,3,4,2,4,5,3,4,4,4,4,1,5,4,5,4,5,5,2,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2937,3,2,2002,3,1,3,5,4,1,1,4,3,4,4,3,5,5,3,4,3,5,3,4,3,5,5,3,3,5,3,5,3,3,3,4,4,4,3,4,5,2,4,4,2,4,5,3,4,3,5
2938,3,2,2003,3,2,4,5,3,3,3,3,5,5,5,1,5,5,4,3,5,5,5,5,5,4,4,5,4,3,2,4,3,2,5,3,4,4,4,5,5,1,5,3,2,3,5,4,5,4,5
2939,3,2,2002,4,1,4,3,5,1,3,5,2,4,5,1,3,5,4,5,3,3,2,4,3,4,4,3,4,2,3,3,5,1,3,4,4,4,5,3,4,1,4,4,4,5,3,5,2,4,4
2940,4,1,2002,4,3,1,3,1,2,4,4,4,4,4,2,4,4,3,2,3,4,4,4,5,5,3,4,4,5,3,5,3,1,4,4,3,5,4,4,3,1,4,2,5,3,4,4,2,4,4


In [34]:
data_b

,birth,grade,gender,school,gpa,gpa_des,livewith,mom_edu,mom_occ,mom_inc,dad_edu,dad_occ,par_neg_reg,par_hard_appr,par_exam_acpt,par_achv_care,par_well_love,par_achv_warm,par_pos_reg,par_lazy_appr_neg,par_exam_acpt_neg,par_achv_care_neg,par_well_love_neg,par_achv_warm_neg,teach_neg_reg,teach_hard_appr,teach_exam_acpt,teach_achv_care,teach_well_like,teach_achv_fnd,teach_pos_reg,teach_hard_appr_neg,teach_exam_acpt_neg,teach_achv_care_neg,teach_well_like_neg,teach_achv_fnd_neg,acad_cond_self_worth,achv_good_feel,achv_bad_feel,achv_affect_feel,achv_value,achv_worth,al_learn,al_learn_like,al_learn_enjoy,al_learn_boring,al_learn_exciting,al_learn_pleasure,al_learn_useful,al_learn_useless,al_learn_waste,al_teach,al_teach_nervous,al_teach_accept,al_teach_comfort,al_teach_respect,al_teach_under,al_teach_care,al_teach_feeling,al_teach_trust,al_class,al_class_nervous
0,2002,3,2,4,5,5,1,5,7,5,4,7,4,4,3,3,5,1,4,5,2,3,4,1,4,2,5,4,4,2,3,3,5,4,5,3,3,3,4,4,2,4,3,3,4,2,3,4,3,3,2,2,1,4,1,5,2,5,2,1,5,4
1,2002,3,2,6,5,4,2,4,4,3,5,7,5,4,5,5,4,3,5,5,4,1,5,2,4,5,3,5,4,2,3,5,4,2,5,1,5,5,4,4,1,4,5,2,3,3,3,3,2,2,1,3,1,4,1,5,1,5,5,2,4,4
2,2002,3,1,6,5,5,1,3,5,4,4,10,2,5,4,3,4,3,2,1,5,4,4,4,3,5,5,3,5,1,4,3,4,5,5,3,5,5,5,3,5,5,3,1,3,1,1,2,4,1,1,4,2,5,2,5,1,5,5,1,5,4
3,2002,3,2,6,5,5,1,4,2,4,4,6,5,4,4,3,3,4,5,4,2,5,3,5,4,3,5,5,5,1,2,4,3,4,5,3,5,5,5,4,1,4,1,1,3,1,2,3,3,1,1,1,1,4,1,5,1,5,4,1,5,4
4,2002,3,1,6,5,5,1,5,7,3,3,1,4,4,4,4,4,1,4,3,5,5,4,4,1,3,3,4,5,2,5,5,5,4,5,3,5,4,5,5,2,3,3,1,1,2,3,4,1,2,3,1,1,5,1,5,1,4,5,1,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2965,2003,2,1,7,5,4,1,3,4,5,4,7,5,4,4,5,4,3,3,3,3,4,5,5,3,4,5,4,4,2,4,4,4,5,5,5,4,5,4,3,2,5,1,2,2,1,1,3,2,2,2,1,2,4,1,5,1,5,5,1,5,4
2966,2002,3,2,1,4,4,1,3,4,4,4,7,4,3,5,3,4,3,3,5,5,4,4,3,5,4,5,4,5,1,4,5,3,5,5,5,5,5,4,5,1,5,2,2,1,1,2,4,1,1,1,2,1,5,1,5,1,5,4,1,5,5
2967,2003,2,1,7,4,4,2,4,4,4,4,4,4,5,5,3,5,5,3,5,5,5,5,5,5,4,4,5,5,1,4,5,5,5,5,4,4,5,5,5,1,5,1,3,2,2,1,5,2,2,2,2,1,4,1,5,1,5,4,1,3,4
2968,2003,2,2,7,5,5,1,3,1,4,3,2,4,4,4,4,5,4,5,4,4,3,5,3,4,4,5,5,4,2,4,5,5,5,5,4,5,4,4,5,2,5,1,1,1,1,1,5,1,1,3,1,1,4,1,5,2,5,4,1,5,3


In [36]:
# ==============================================================================
# CELL 3: XỬ LÝ GIÁ TRỊ THIẾU (MISSING VALUES)
# ==============================================================================

print("Kiểm tra giá trị thiếu trước khi xử lý:")
print("Dataset A:", data_a.isnull().sum().sum(), "ô bị thiếu")
print("Dataset B:", data_b.isnull().sum().sum(), "ô bị thiếu")


Kiểm tra giá trị thiếu trước khi xử lý:
Dataset A: 0 ô bị thiếu
Dataset B: 0 ô bị thiếu


In [37]:
# ==============================================================================
# CELL 4: CHUẨN HÓA DỮ LIỆU VÀ HUẤN LUYỆN MÔ HÌNH PHÂN CỤM
# ==============================================================================
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import joblib

# --- Xử lý Dataset A ---
# Chỉ lấy các cột tâm lý để phân cụm
psychological_cols_a = data_a.columns[4:]
data_a_psych = data_a[psychological_cols_a]

scaler_a = StandardScaler()
data_a_scaled = scaler_a.fit_transform(data_a_psych)

# Huấn luyện mô hình K-Means cho Dataset A
# Giả sử chúng ta muốn tìm 4 "kiểu" động lực
kmeans_a = KMeans(n_clusters=4, random_state=42, n_init='auto')
kmeans_a.fit(data_a_scaled)
print("✅ Huấn luyện mô hình cho Dataset A thành công.")

# --- Xử lý Dataset B ---
# Chỉ lấy các cột tâm lý để phân cụm
psychological_cols_b = data_b.columns[12:]
data_b_psych = data_b[psychological_cols_b]

scaler_b = StandardScaler()
data_b_scaled = scaler_b.fit_transform(data_b_psych)

# Huấn luyện mô hình K-Means cho Dataset B
# Giả sử chúng ta muốn tìm 4 "kiểu" gắn kết
kmeans_b = KMeans(n_clusters=4, random_state=42, n_init='auto')
kmeans_b.fit(data_b_scaled)
print("✅ Huấn luyện mô hình cho Dataset B thành công.")


# --- Lưu các mô hình và scaler đã huấn luyện ra file ---
joblib.dump(kmeans_a, 'model_a_motivation.joblib')
joblib.dump(scaler_a, 'scaler_a_motivation.joblib')
joblib.dump(kmeans_b, 'model_b_engagement.joblib')
joblib.dump(scaler_b, 'scaler_b_engagement.joblib')

print("\n💾 Đã lưu thành công 4 files: 2 model và 2 scaler.")
print("   Các file này sẽ được sử dụng trong notebook chính.")

✅ Huấn luyện mô hình cho Dataset A thành công.
✅ Huấn luyện mô hình cho Dataset B thành công.

💾 Đã lưu thành công 4 files: 2 model và 2 scaler.
   Các file này sẽ được sử dụng trong notebook chính.


In [39]:
# ==============================================================================
# CELL 5: PHÂN TÍCH VÀ DIỄN GIẢI KẾT QUẢ PHÂN CỤM
# ==============================================================================

# Gán nhãn cụm cho từng học sinh vào DataFrame gốc
data_a['motivation_cluster'] = kmeans_a.labels_
data_b['engagement_cluster'] = kmeans_b.labels_

# Phân tích đặc điểm trung bình của từng cụm
print("--- Phân tích Cụm Động lực (Dataset A) ---")
cluster_analysis_a = data_a.groupby('motivation_cluster')[psychological_cols_a].mean().T
print(cluster_analysis_a)

print("\n--- Phân tích Cụm Gắn kết (Dataset B) ---")
cluster_analysis_b = data_b.groupby('engagement_cluster')[psychological_cols_b].mean().T
print(cluster_analysis_b)

--- Phân tích Cụm Động lực (Dataset A) ---
motivation_cluster                  0         1         2         3
FRE_ACT_Mental_Health        2.421495  2.206369  2.031311  2.347222
FRE_ACT_Physical_Health      3.516822  3.815287  2.720157  3.461806
FRE_ACT_Environment          3.540187  2.433121  2.816047  3.512153
FRE_ACT_Education            2.587850  2.620382  2.346380  2.645833
FRE_ACT_Gender_Equity        2.426168  2.336306  2.448141  2.300347
FRE_ACT_Poverty_Eradication  3.489720  2.853503  2.113503  3.451389
FRE_ACT_Peace_Justice        3.877570  3.129936  2.618395  3.847222
PERF_LEV_EC1                 4.464486  3.073885  2.091977  2.831597
PERF_LEV_EC1                 4.023364  3.456051  1.847358  4.017361
PERF_LEV_EC1                 4.464486  3.073885  2.091977  2.831597
PERF_LEV_EC1                 4.023364  3.456051  1.847358  4.017361
PERF_LEV_EC1                 4.464486  3.073885  2.091977  2.831597
PERF_LEV_EC1                 4.023364  3.456051  1.847358  4.017361
PERF_

In [40]:
print("\n--- Phân tích Cụm Gắn kết (Dataset B) ---")
cluster_analysis_b = data_b.groupby('engagement_cluster')[psychological_cols_b].mean().T
print(cluster_analysis_b)


--- Phân tích Cụm Gắn kết (Dataset B) ---
engagement_cluster           0         1         2         3
par_neg_reg           3.739615  3.711697  3.757664  3.790159
par_hard_appr         4.020263  4.041186  4.385401  4.030391
par_exam_acpt         4.047619  3.978583  3.951825  4.024602
par_achv_care         3.884498  3.892916  4.042336  3.821997
par_well_love         4.013171  4.062603  3.576642  4.017366
par_achv_warm         4.047619  4.049423  3.281752  4.120116
par_pos_reg           3.976697  3.955519  3.922628  4.010130
par_lazy_appr_neg     4.027356  4.041186  3.824818  4.063676
par_exam_acpt_neg     4.054711  4.067545  3.934307  4.011577
par_achv_care_neg     4.173252  4.177924  3.548905  4.112880
par_well_love_neg     4.173252  4.151565  3.395620  4.218524
par_achv_warm_neg     4.118541  4.153213  3.840876  4.059334
teach_neg_reg         4.136778  4.131796  3.884672  4.128799
teach_hard_appr       4.213779  4.252059  4.072993  4.195369
teach_exam_acpt       4.329281  4.344316  